# Chapter 8: Transformations

## Beta and Gamma distributions

The Beta and Gamma distributions are programmed into R.

- `dbeta`, `pbeta`, `rbeta`: To evaluate the $\text{Beta}(a,b)$ PDF or CDF at $x$, we use `dbeta(x,a,b)` and `pbeta(x,a,b)`. To generate $n$ realizations from the $\text{Beta}(a,b)$ distribution, we use `rbeta(n,a,b)`.
- `dgamma`, `pgamma`, `rgamma`: To evaluate the $\text{Gamma}(a,\lambda)$ PDF or CDF at $x$, we use `dgamma(x,a,lambda)` or `pgamma(x,a,lambda)`. To generate $n$ realizations from the $\text{Gamma}(a,\lambda)$ distribution, we use `rgamma(n,a,lambda)`.

For example, we can check that the $\text{Gamma}(3,2)$ distribution has mean $3/2$ and variance $3/4$. To do this, we generate a large number of $\text{Gamma}(3,2)$ random variables using `rgamma`, then compute their `mean` and `var`:

In [ ]:
y <- rgamma(10^5,3,2)
mean(y)
var(y)

Did you get values that were close to 1.5 and 0.75, respectively?

## Convolution of Uniforms

Using  R, we can quickly verify that for $X,Y \overset{\text{i.i.d.}}{\sim} \text{Unif}(0,1)$, the distribution of $T = X + Y$ is triangular in shape:

In [ ]:
x <- runif(10^5)
y <- runif(10^5)
hist(x+y)

The histogram looks like an ascending and then descending staircase, a discrete approximation to a triangle.

## Bayes' billiards

In the Bayes' billiards story, we have $n$ white balls and $1$ gray ball, throw them onto the unit interval completely at random, and count the number of white balls to the left of the gray ball. Letting $p$ be the position of the gray ball and $X$ be the number of white balls to the left of the gray ball, we have
$$\begin{gather*} p \sim \text{Unif}(0,1) \\ X | p \sim \text{Bin}(n,p). \end{gather*}$$

By performing this experiment a large number of times, we can verify the results we derived in this chapter about the marginal PMF of $X$ and the posterior PDF of $p$ given $X=x$. We'll let the number of simulations be called `nsim`, to avoid a name conflict with the number of white balls, `n`, which we set equal to $10$:

In [ ]:
nsim <- 10^5
n <- 10

We simulate $10^5$ values of $p$, then simulate $10^5$ values from the conditional distribution of $X$ given $p$:

In [ ]:
p <- runif(nsim)
x <- rbinom(nsim,n,p)

Notice that we feed the entire vector `p` into `rbinom`. This means that the first element of `x` is generated using the first element of `p`, the second element of `x` is generated using the second element of `p`, and so forth. Thus, conditional on a particular element of `p`, the corresponding element of `x` is Binomial, but the elements of `p` are themselves Uniform, exactly as the model specifies.

According to the Bayes’ billiards argument, the marginal distribution of $X$ should be Discrete Uniform on the integers $0$ through $n$. Is this in fact the case? We can make a histogram of `x` to check! Because the distribution of $X$ is discrete, we tell R to make the histogram breaks at $−0.5,0.5,1.5,\ldots$ so that each bar is centered at an integer value:

In [ ]:
hist(x,breaks=seq(-0.5,n+0.5,1))

Indeed, all the histogram bars are approximately equal in height, consistent with a Discrete Uniform distribution.

Now for the posterior distribution of $p$ given $X=x$. Conditioning is very simple in R. To consider only the simulated values of $p$ where the value of $X$ was 3, we use square brackets, like this: `p[x==3]`. In particular, we can create a histogram of these values to see what the posterior distribution of $p$ given $X= 3$ looks like; try `hist(p[x==3])`.

According to the Beta-Binomial conjugacy result, the true posterior distribution is $p|X=  3\sim \text{Beta}(4,8)$. We can plot the histogram of `p[x==3]` next to a histogram of simulated values from the $\text{Beta}(4,8)$ distribution to confirm that they look similar:

In [ ]:
par(mfrow=c(1,2))
hist(p[x==3])
hist(rbeta(10^4,4,8))

The first line tells R we want two side-by-side plots, and the second and third lines create the histograms.

## Simulating order statistics

Simulating order statistics in R is easy: we simply simulate i.i.d. r.v.s and use `sort` to sort them in increasing order. For example,

In [ ]:
sort(rnorm(10))

produces one realization of $X_{(1)},\ldots,X_{(10)}$, where $X_1,\ldots,X_{10}$ are i.i.d. $\mathcal N(0,1)$.

If we want to plot a histogram of realizations of, say, $X_{(9)}$, we'll need to use `replicate`:

In [ ]:
order_stats <- replicate(10^4, sort(rnorm(10)))

This creates a matrix, `order_stats`, with 10 rows. The $i\text{th}$ row of the matrix contains $10^4$ realizations of $X_{(i)}$. Now we can create a histogram of $X_{(9)}$, simply by selecting row $9$ of the matrix:

In [ ]:
x9 <- order_stats[9,]
hist(x9)

We can also compute summaries like `mean(x9)` and `var(x9)`.